In [9]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from typing import Any

In [10]:
config: dict = {
    'gpu': 'mps',
    'data_dir': './train_data.npy',
    'label_dir': './train_labels.npy',
    'batch_size': 16,
    'train_ratio':.8
}

In [11]:
class LOBDataset(Dataset):
    def __init__(self,
                 data: torch.tensor, 
                 labels:torch.tensor,
                #  is_Train: bool = True, 
                 ) -> None:
        super().__init__()
        # if is_train is True:
        self.data = data
        self.labels= labels
        # 都是float32的数据了

    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self, index) -> Any:
        return self.data[index].to(config['gpu']), self.labels[index].to(config['gpu'])

    # TODO: 划分训练集和测试集

In [12]:
def get_dataloader(batch_size:int, train_ratio:float, label_idx: int = 0,
                   )->tuple[DataLoader,DataLoader]:
    '''read data from file and return a tuple of DataLoader'''
    data_dir = config['data_dir']
    label_dir = config['label_dir']

    data: torch.tensor = torch.from_numpy(np.load(data_dir))
    labels: torch.tensor = torch.from_numpy(np.load(label_dir))
    # TODO 暴力切一刀是有问题的
    train_data = data[:round(train_ratio * data.shape[0])]
    train_labels = labels[:round(train_ratio * data.shape[0]),label_idx]

    valid_data = data[round(train_ratio * data.shape[0]):]
    valid_labels = labels[round(train_ratio * data.shape[0]):,label_idx]

    train_iter = DataLoader(LOBDataset(train_data, train_labels),batch_size=batch_size, shuffle=False)
    valid_iter = DataLoader(LOBDataset(valid_data, valid_labels),batch_size=batch_size, shuffle=False)
    return train_iter, valid_iter

In [13]:
train_iter, valid_iter = get_dataloader(batch_size=config['batch_size'], train_ratio=config['train_ratio'])
m = next(iter(train_iter))
print(m[0].dtype)
print(m[0].shape)

torch.float32
torch.Size([16, 26])
